# Part 7: Build an Encrypted, Decentralized Ledger

In this section, we're going to add a slight extension to the key-value database we made in Part 6 and instead modify it to be a very performant decentralized ledger. 

At present, you put all of your money into a single bank account at a time. This bank always knows your balance and has the ability to send/receive money on your behalf (perhaps even without your permission!!). 

We're going to design a simple prototype of an (extremely performant) database in which your account is with MULTIPLE banks, but NONE of them can see your balance, and transactions can only occur if all banks work together (they would have to collude to spend on your behalf).

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

# Step 1: A Simple Ledger

Before we can attempt to build an encrypted, decentralized ledger, we must first build a simple, decrypted one. This is simple enough. We have a list of starting values, and each transaction changes those values.

In [5]:
import syft as sy

In [15]:
num_accounts = 10

transactions = list()

initial_balances = sy.ones(num_accounts).long() * 100
transactions.append(initial_balances)

def transact(amt=10,frm=0,to=1):
    record = sy.zeros(num_accounts).long()
    record[frm] = -amt
    record[to] = amt
    transactions.append(record)
    

In [16]:
transact(5,0,1)

In [17]:
transactions

[
  100
  100
  100
  100
  100
  100
  100
  100
  100
  100
 [torch.LongTensor of size 10], 
 -5
  5
  0
  0
  0
  0
  0
  0
  0
  0
 [torch.LongTensor of size 10]]

## Getting Your Balance

And if you want to calculate the current balance for everyone, you just sum up all the records!

In [18]:
balance = transactions[0]
for t in transactions[1:]:
    balance = balance + t

In [20]:
balance


  95
 105
 100
 100
 100
 100
 100
 100
 100
 100
[torch.LongTensor of size 10]

### A Bit of Object Orientied Organization

Now let's put these features into a proper class.

In [44]:
class SimpleLedger():

    def __init__(self, num_accounts=10, starting_balance=100):
        
        self.transactions = list()
        self.num_accounts = num_accounts
        
        initial_balances = sy.ones(num_accounts).long() * starting_balance
        self.transactions.append(initial_balances)
        
    def transact(self, amt=10,frm=0,to=1):
        record = sy.zeros(self.num_accounts).long()
        record[frm] = -amt
        record[to] = amt
        self.transactions.append(record)
        
    def compute_balances(self):
        balance = self.transactions[0]
        for t in self.transactions[1:]:
            balance = balance + t

        return balance

In [45]:
ledger = SimpleLedger()

In [46]:
ledger.transact(10, 0, 1)

In [47]:
ledger.compute_balances()


  90
 110
 100
 100
 100
 100
 100
 100
 100
 100
[torch.LongTensor of size 10]

# Step 2: Checking Balance Values

So our ledger is great, but unfortunately we can transact negative money! We need to be able to deny transactions if the balance for them wasn't available. For this we need two things:

- 1) maintain a running list of balances
- 2) deny transactions which force balances to become negative

In [67]:
class SimpleLedger():

    def __init__(self, num_accounts=10, starting_balance=100):
        
        self.transactions = list()
        self.num_accounts = num_accounts
        
        initial_balances = sy.ones(num_accounts).long() * starting_balance
        self.transactions.append(initial_balances)
        
        self.running_balance = self.compute_balances()
        
    def transact(self, amt=10,frm=0,to=1):
        record = sy.zeros(self.num_accounts).long()
        record[frm] = -amt
        record[to] = amt
        
        candidate_balance = self.running_balance + record
        neg_check = 1 - (candidate_balance < 0).sum()
        
        record = record * neg_check
        
        self.transactions.append(record)
        self.running_balance = self.running_balance + record
        
    def compute_balances(self):
        balance = self.transactions[0]
        for t in self.transactions[1:]:
            balance = balance + t
            
        return balance

In [68]:
ledger = SimpleLedger()

In [69]:
# valid transactions work fine
ledger.transact(100, 0, 1)
ledger.compute_balances()


   0
 200
 100
 100
 100
 100
 100
 100
 100
 100
[syft.core.frameworks.torch.tensor.LongTensor of size 10]

In [70]:
# invalid transactions don't run!
ledger.transact(100, 0, 1)
ledger.compute_balances()


   0
 200
 100
 100
 100
 100
 100
 100
 100
 100
[syft.core.frameworks.torch.tensor.LongTensor of size 10]

# Step 3: Encrypting / Decentralizing the Ledger

And now all we need to do is SMPC share all of the tensors involved and we have an encrypted, decentralized ledger!

In [107]:
class DecentralizedLedger():

    def __init__(self, *owners, num_accounts=10, starting_balance=100):
        
        self.transactions = list()
        self.owners = owners
        self.num_accounts = num_accounts
        
        initial_balances = sy.ones(num_accounts).long() * starting_balance
        initial_balances = initial_balances.share(*self.owners)
        self.transactions.append(initial_balances)
        
        # cache zeros for use in comparison
        self.zeros = initial_balances - initial_balances
        
        self.running_balance = self.compute_balances()
        
    def transact(self, amt=10,frm=0,to=1):
        record = sy.zeros(self.num_accounts).long()
        record[frm] = -amt
        record[to] = amt
        
        record = record.share(*self.owners)
        
        candidate_balance = self.running_balance + record
        neg_check = 1 - (candidate_balance < self.zeros).sum(0).expand(self.num_accounts)
        
        record = record * neg_check
        
        self.transactions.append(record)
        self.running_balance = self.running_balance + record
        
    def compute_balances(self):
        balance = self.transactions[0]
        for t in self.transactions[1:]:
            balance = balance + t
            
        return balance

In [110]:
import syft as sy
hook = sy.TorchHook()

jp_norgan = sy.VirtualWorker(id="jp_norgan")
ubz = sy.VirtualWorker(id="ubz")

ledger = DecentralizedLedger(jp_norgan, ubz)

# valid transactions work fine
ledger.transact(100, 0, 1)
ledger.compute_balances().get()


   0
 200
 100
 100
 100
 100
 100
 100
 100
 100
[syft.core.frameworks.torch.tensor.LongTensor of size 10]

In [111]:
# spending too much from an account will cancel the transaction
ledger.transact(100, 0, 1)
ledger.compute_balances().get()


   0
 200
 100
 100
 100
 100
 100
 100
 100
 100
[syft.core.frameworks.torch.tensor.LongTensor of size 10]

# Success!!

And there you have it, we now have created a banking system hosted by two banks JP Norgan and UBZ. However, neither JP Norgan or UBZ can see anyone's balances OR unilaterally transact. However, they can still check to ensure that transactions are valid (only spending money which is actually in people's accounts).

# Congratulations!!! - Time to Join the Community!

Congraulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on Github

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft Github Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for github issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)